# Trabajo de caso real Arquitectura Big Data

Se trata de un conjunto de supermercados que tienen varios años de datos acumulados de productos de limpieza del hogar.
Realizan promociones de forma permanente y quieren poder predecir el volumen de ventas según las promociones que realizan.


### Importamos las librerías necesarias

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración para visualizaciones
%matplotlib inline
sns.set(style="whitegrid")

### Cargamos los datos

In [2]:
locales = pd.read_csv('VMI_Locales-20190712.txt', sep='\t', encoding='ISO-8859-1', header=None)
movimientos = pd.read_csv('VMI_Movimientos-20190716.txt', sep='\t', encoding='ISO-8859-1', header=None)
promociones = pd.read_csv('VMI_Promociones-20190715.txt', sep='\t', encoding='ISO-8859-1', header=None)
prov_productos = pd.read_csv('VMI_ProvProdu-20190712.txt', sep='\t', encoding='ISO-8859-1', header=None)

In [3]:
locales.columns = ['ID Cadena', 'DescCadena', 'ID Region', 'DescRegion', 'ID Local/CD', 'DescLocal', 'Latitud', 'Longitud', 'PD']
movimientos.columns = ['ID Local/CD', 'ID producto', 'Fecha de la transaccion', 'Stock en unidades', 'Venta dia anterior en unidades', 'Precio de venta unitario', 'CD que abastece', 'RolLocal']
promociones.columns = ['ID Local/CD', 'ID producto', 'Fecha-desde', 'Fecha-Hasta', 'Total ventas en Unidades', 'Nro. Promocion', 'Fecha de la transaccion', 'Porc_Descuento']



## EDA Dataset Locales

**ID Cadena** ->	Identificador interno de Cadena  
**DescCadena** ->	Descripción de la Cadena  
**IdRegion** ->	    Identificador interno de Región  
**DescRegion** ->  	Descripción de la Región  
**ID Local/CD** ->	Identificador interno de calle  
**DescLocal** ->	Nombre de la calle  
**Latitud (Y)** ->	Latitud	Ubicación 'Y' de la calle  
**Longitud (X)** ->	Longitud	Ubicación 'X' de la calle  
**PD** ->	        Indicador PD (si es o no es un PD, si no es un PD es un Local) PD= Punto de Distribución = CD  = Centro de Distribucion

In [4]:
locales.head()

,ID Cadena,DescCadena,ID Region,DescRegion,ID Local/CD,DescLocal,Latitud,Longitud,PD
0,1,Plaza Vea,1,Buenos Aires,9693,9693 - P Vea Villa Urquiza.,NaN,NaN,N
1,1,Plaza Vea,1,Buenos Aires,9694,9694 - P Vea JB Justo.,NaN,NaN,N
2,1,Plaza Vea,1,Buenos Aires,9695,695 - P Vea Acoyte.,0.0,0.0,N
3,1,Plaza Vea,1,Buenos Aires,9699,699 - P Vea Rivadavia.,0.0,0.0,N
4,1,Plaza Vea,1,Buenos Aires,9889,889 - Plaza Vea Carpa de liqui,NaN,NaN,N


Observamos que en la columna de Latitud y Longitud aparecen valores con 0.0, o con valores NaN.

In [5]:
locales.shape

(175, 9)

In [6]:
locales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID Cadena    175 non-null    int64  
 1   DescCadena   175 non-null    object 
 2   ID Region    175 non-null    int64  
 3   DescRegion   175 non-null    object 
 4   ID Local/CD  175 non-null    int64  
 5   DescLocal    175 non-null    object 
 6   Latitud      140 non-null    float64
 7   Longitud     140 non-null    float64
 8   PD           175 non-null    object 
dtypes: float64(2), int64(3), object(4)
memory usage: 12.4+ KB


Comprobamos el número de valores NaN y 0.0

In [7]:
locales.isnull().sum()

ID Cadena       0
DescCadena      0
ID Region       0
DescRegion      0
ID Local/CD     0
DescLocal       0
Latitud        35
Longitud       35
PD              0
dtype: int64

In [8]:
# Detectando locales con latitud y longitud NaN
locales_con_nan = locales[locales['Latitud'].isna() & (locales['Longitud'].isna())]

locales_con_nan

,ID Cadena,DescCadena,ID Region,DescRegion,ID Local/CD,DescLocal,Latitud,Longitud,PD
0,1,Plaza Vea,1,Buenos Aires,9693,9693 - P Vea Villa Urquiza.,NaN,NaN,N
1,1,Plaza Vea,1,Buenos Aires,9694,9694 - P Vea JB Justo.,NaN,NaN,N
4,1,Plaza Vea,1,Buenos Aires,9889,889 - Plaza Vea Carpa de liqui,NaN,NaN,N
7,2,Disco,1,Buenos Aires,285,953 - Venado Tuerto,NaN,NaN,N
9,2,Disco,1,Buenos Aires,961,961 - Av. Santa F,NaN,NaN,N
10,2,Disco,1,Buenos Aires,1190,6190 - BT SAF (Importaciones).,NaN,NaN,S
11,2,Disco,1,Buenos Aires,1914,3914 - Interno SLL BsAs 217.,NaN,NaN,S
67,4,Vea,1,Buenos Aires,248,917 - Mataderos II,NaN,NaN,N
69,4,Vea,1,Buenos Aires,250,168 - Lope de Vega,NaN,NaN,N
71,4,Vea,1,Buenos Aires,255,995 - Alvarez Jonte,NaN,NaN,N


In [9]:
locales.eq(0.0).sum()

ID Cadena       0
DescCadena      0
ID Region       0
DescRegion      0
ID Local/CD     0
DescLocal       0
Latitud        14
Longitud       14
PD              0
dtype: int64

Observamos que hay 14 locales con la latitud y longitud en 0.0. más 35 locales con la latitud y longitud con valor NaN. De 175 locales. 

In [10]:
# Detectando locales con latitud y longitud en 0.0
locales_con_cero = locales[(locales['Latitud'] == 0.0) & (locales['Longitud'] == 0.0)]

locales_con_cero

,ID Cadena,DescCadena,ID Region,DescRegion,ID Local/CD,DescLocal,Latitud,Longitud,PD
2,1,Plaza Vea,1,Buenos Aires,9695,695 - P Vea Acoyte.,0.0,0.0,N
3,1,Plaza Vea,1,Buenos Aires,9699,699 - P Vea Rivadavia.,0.0,0.0,N
16,2,Disco,1,Buenos Aires,9007,7 - Pte. Per¢n.,0.0,0.0,N
55,2,Disco,1,Buenos Aires,9205,205 - Carniceria Central.,0.0,0.0,N
63,2,Disco,1,Buenos Aires,9639,639 - Pinocho VII.,0.0,0.0,N
70,4,Vea,1,Buenos Aires,252,983 - Miramar,0.0,0.0,N
80,4,Vea,1,Buenos Aires,674,674 - Resistencia,0.0,0.0,N
134,9,Jumbo,1,Buenos Aires,305,305 - DEPOSITO CBN GROUP,0.0,0.0,S
153,9,Jumbo,1,Buenos Aires,5276,5276 - Jumbo Arenales,0.0,0.0,N
155,9,Jumbo,1,Buenos Aires,5299,5299 - Dep¢sito Tortuguitas,0.0,0.0,S


In [11]:
locales_con_coordenadas = locales[(locales['Latitud'].notna()) & (locales['Latitud'] != 0.0) &
                                   (locales['Longitud'].notna()) & (locales['Longitud'] != 0.0)]
locales_con_coordenadas

,ID Cadena,DescCadena,ID Region,DescRegion,ID Local/CD,DescLocal,Latitud,Longitud,PD
5,2,Disco,1,Buenos Aires,247,916 - SM 916 Gorostiaga,-34.565259,-58.436583,N
6,2,Disco,1,Buenos Aires,257,61 - Tortuguitas,-34.446043,-58.745098,N
8,2,Disco,1,Buenos Aires,740,740 - Canning,-34.853221,-58.502247,N
12,2,Disco,1,Buenos Aires,9001,1 - San Isidro.,-34.471531,-58.515350,N
13,2,Disco,1,Buenos Aires,9002,2 - Talcahuano.,-34.596256,-58.385701,N
...,...,...,...,...,...,...,...,...,...
151,9,Jumbo,1,Buenos Aires,5263,5263 - Jumbo Juan B. Justo,-34.609829,-58.465505,N
154,9,Jumbo,1,Buenos Aires,5277,5277 - Jumbo-Comodoro,-45.872649,-67.496036,N
159,9,Jumbo,1,Buenos Aires,5626,5626 - Jumbo Pacheco Novo,-34.453186,-58.630924,N
160,9,Jumbo,1,Buenos Aires,5961,5961 - Jumbo Av. Santa F,-34.577317,-58.428886,N


De momento vamos a mantener las lineas con latitud y longitud en 0.0 y valores NaN. 

A continuación vamos a ver si existen datos duplicados.

In [12]:
# Contar duplicados basados en 'ID Local/CD'
duplicados = locales.duplicated(subset='ID Local/CD', keep=False).sum()
print(f'Número de duplicados basados en ID Local/CD: {duplicados}')


Número de duplicados basados en ID Local/CD: 0


En la columna PD he detectado que tiene valor 'N' y 'S' teniendo en cuenta la documentación, los valores nos están indicando con una N que NO es un Punto de distribución y con una S que SI es un Punto de distribución..

In [13]:
locales['PD'].unique()

array(['N', 'S'], dtype=object)

In [14]:
locales['PD'].value_counts()

PD
N    169
S      6
Name: count, dtype: int64

Después de realizar un conteo, observamos que hay 6 locales que SI son un punto de Distribución y el resto no lo son.

In [59]:
# Guardar el DataFrame en un archivo CSV
locales.to_csv('locales_clean.csv', index=False)

## EDA Dataset Movimientos

In [15]:
movimientos.head()

,ID Local/CD,ID producto,Fecha de la transaccion,Stock en unidades,Venta dia anterior en unidades,Precio de venta unitario,CD que abastece,RolLocal
0,242,228217,20160715,24.0,0.0,13.63,0,SI
1,242,228217,20160716,24.0,0.0,13.63,0,SI
2,242,228217,20160717,24.0,0.0,13.63,0,SI
3,242,228217,20160718,24.0,0.0,13.63,0,SI
4,242,228217,20160719,24.0,0.0,13.63,0,SI


In [16]:
movimientos.shape

(8347817, 8)

In [17]:
movimientos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8347817 entries, 0 to 8347816
Data columns (total 8 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   ID Local/CD                     int64  
 1   ID producto                     int64  
 2   Fecha de la transaccion         int64  
 3   Stock en unidades               float64
 4   Venta dia anterior en unidades  float64
 5   Precio de venta unitario        float64
 6   CD que abastece                 int64  
 7   RolLocal                        object 
dtypes: float64(3), int64(4), object(1)
memory usage: 509.5+ MB


Observamos que:  
1.'Venta día anterior en unidades' está en formato decimal en lugar de número entero.  
2.'Precio de venta unitario' tiene valores 0.0  
3.'Fecha de la transaccion' no está en formato fecha y además según documentación es 1 día menos que la fecha actual. 

### Cambiamos el formato de la columna 'Venta dia anterior en unidades' a formato número entero

In [18]:
movimientos['Venta dia anterior en unidades'] = movimientos['Venta dia anterior en unidades'].astype(int)

### Cambiamos el formato de la columna 'Fecha de la transaccion' a formato fecha


In [19]:
movimientos['Fecha de la transaccion'] = pd.to_datetime(movimientos['Fecha de la transaccion'], format='%Y%m%d', errors='coerce')


In [20]:
movimientos.dtypes

ID Local/CD                                int64
ID producto                                int64
Fecha de la transaccion           datetime64[ns]
Stock en unidades                        float64
Venta dia anterior en unidades             int32
Precio de venta unitario                 float64
CD que abastece                            int64
RolLocal                                  object
dtype: object

In [21]:
movimientos

,ID Local/CD,ID producto,Fecha de la transaccion,Stock en unidades,Venta dia anterior en unidades,Precio de venta unitario,CD que abastece,RolLocal
0,242,228217,2016-07-15,24.0,0,13.63,0,SI
1,242,228217,2016-07-16,24.0,0,13.63,0,SI
2,242,228217,2016-07-17,24.0,0,13.63,0,SI
3,242,228217,2016-07-18,24.0,0,13.63,0,SI
4,242,228217,2016-07-19,24.0,0,13.63,0,SI
...,...,...,...,...,...,...,...,...
8347812,9977,511326,2019-07-10,9.0,0,100.82,9217,SI
8347813,9977,511326,2019-07-11,9.0,0,100.82,9217,SI
8347814,9977,511326,2019-07-12,9.0,0,100.82,9217,SI
8347815,9977,511326,2019-07-13,9.0,0,100.82,9217,SI


In [22]:
movimientos.isnull().sum()

ID Local/CD                       0
ID producto                       0
Fecha de la transaccion           0
Stock en unidades                 0
Venta dia anterior en unidades    0
Precio de venta unitario          0
CD que abastece                   0
RolLocal                          0
dtype: int64

In [23]:
movimientos.eq(0.0).sum()

ID Local/CD                             0
ID producto                             0
Fecha de la transaccion                 0
Stock en unidades                  657428
Venta dia anterior en unidades    6722807
Precio de venta unitario             9859
CD que abastece                   3425090
RolLocal                                0
dtype: int64

In [58]:
# Guardar el DataFrame en un archivo CSV
movimientos.to_csv('movimientos_clean.csv', index=False)

## EDA Dataset Promociones

In [24]:
promociones.head()

,ID Local/CD,ID producto,Fecha-desde,Fecha-Hasta,Total ventas en Unidades,Nro. Promocion,Fecha de la transaccion,Porc_Descuento
0,242,228217,20161023,20161030,0.98,436830,20161005,7.0
1,242,228217,20161023,20161030,0.98,436830,20161006,7.0
2,242,228217,20161023,20161030,0.98,436830,20161007,7.0
3,242,228217,20161023,20161030,0.98,436830,20161008,7.0
4,242,228217,20161023,20161030,0.98,436830,20161009,7.0


In [25]:
promociones.shape

(285253, 8)

In [26]:
promociones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285253 entries, 0 to 285252
Data columns (total 8 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID Local/CD               285253 non-null  int64  
 1   ID producto               285253 non-null  int64  
 2   Fecha-desde               285253 non-null  int64  
 3   Fecha-Hasta               285253 non-null  int64  
 4   Total ventas en Unidades  285253 non-null  float64
 5   Nro. Promocion            285253 non-null  int64  
 6   Fecha de la transaccion   285253 non-null  int64  
 7   Porc_Descuento            285253 non-null  float64
dtypes: float64(2), int64(6)
memory usage: 17.4 MB


Observamos que:  
1.'Total ventas en Unidades' está en formato decimal en lugar de número entero.  
2. Fecha-desde',Fecha-Hasta' y 'Fecha de la transaccion' no están en formato fecha.  
3. 'Total ventas en Unidades' tiene valores está en formato decimal.

#### Cambiamos el formato de la columna 'Total ventas en Unidades' a formato número entero


In [27]:
promociones['Total ventas en Unidades'] = promociones['Total ventas en Unidades'].astype(int)

#### Cambiamos el formato de la columna 'Fecha de la transaccion', 'Fecha-desde' y 'Fecha-Hasta' a formato fecha 

In [28]:
promociones['Fecha de la transaccion'] = pd.to_datetime(promociones['Fecha de la transaccion'], format='%Y%m%d', errors='coerce')

In [29]:
promociones['Fecha-desde'] = pd.to_datetime(promociones['Fecha-desde'], format='%Y%m%d', errors='coerce')

In [30]:
promociones['Fecha-Hasta'] = pd.to_datetime(promociones['Fecha-Hasta'], format='%Y%m%d', errors='coerce')

In [31]:
promociones.dtypes

ID Local/CD                          int64
ID producto                          int64
Fecha-desde                 datetime64[ns]
Fecha-Hasta                 datetime64[ns]
Total ventas en Unidades             int32
Nro. Promocion                       int64
Fecha de la transaccion     datetime64[ns]
Porc_Descuento                     float64
dtype: object

In [32]:
promociones

,ID Local/CD,ID producto,Fecha-desde,Fecha-Hasta,Total ventas en Unidades,Nro. Promocion,Fecha de la transaccion,Porc_Descuento
0,242,228217,2016-10-23,2016-10-30,0,436830,2016-10-05,7.0
1,242,228217,2016-10-23,2016-10-30,0,436830,2016-10-06,7.0
2,242,228217,2016-10-23,2016-10-30,0,436830,2016-10-07,7.0
3,242,228217,2016-10-23,2016-10-30,0,436830,2016-10-08,7.0
4,242,228217,2016-10-23,2016-10-30,0,436830,2016-10-09,7.0
...,...,...,...,...,...,...,...,...
285248,9977,474300,2016-08-16,2016-08-23,12,428910,2016-08-17,7.0
285249,9977,474300,2016-08-16,2016-08-23,12,428910,2016-08-18,7.0
285250,9977,474300,2016-08-16,2016-08-23,12,428910,2016-08-19,7.0
285251,9977,474300,2016-08-16,2016-08-23,12,428910,2016-08-20,7.0


In [57]:
# Guardar el DataFrame en un archivo CSV
promociones.to_csv('promociones_clean.csv', index=False)

## EDA Dataset Prov_Productos

In [33]:
prov_productos.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,1,252,Reckitt Benckiser Argentina S.A.,1,Almacn,4,Limpieza,3,Ba¤o y Hogar,4,Desodorantes de ambientes,1,Desodorantes en aerosol,27003,273920,REFILL CITRICO OIL ESCENCE AIR WICK CITRICOS ...,1,2019-07-12 11:04:07.967
1,1,252,Reckitt Benckiser Argentina S.A.,1,Almacn,4,Limpieza,3,Ba¤o y Hogar,4,Desodorantes de ambientes,1,Desodorantes en aerosol,27004,273921,FRESHMATIC AIRWICK LAVANDA REPUESTO 250.00 MLT,1,2019-07-12 11:04:07.967
2,1,252,Reckitt Benckiser Argentina S.A.,1,Almacn,4,Limpieza,3,Ba¤o y Hogar,4,Desodorantes de ambientes,1,Desodorantes en aerosol,27005,284012,AIR WICK CITRICO FULL REP.GRATIS AIR WICK AIR ...,1,2019-07-12 11:04:07.967
3,1,252,Reckitt Benckiser Argentina S.A.,1,Almacn,4,Limpieza,3,Ba¤o y Hogar,4,Desodorantes de ambientes,1,Desodorantes en aerosol,27012,359572,MAGNOLIA Y CHERRY AIR WICK REPUESTO 175.00 GR,1,2019-07-12 11:04:07.967
4,1,252,Reckitt Benckiser Argentina S.A.,1,Almacn,4,Limpieza,3,Ba¤o y Hogar,4,Desodorantes de ambientes,1,Desodorantes en aerosol,27007,415268,FRESHMATIC BOSQUE MµGICO APARATO AIR WICK LUSH...,1,2019-07-12 11:04:07.967


In [34]:
prov_productos.shape

(82, 18)

Observamos que la lectura de los datos ha sido incorrecta, ha puesto todos los datos en una única columna. Deberíamos de 
hacer una lectura de los datos y separarlos en las columnas correspondientes.

#### Primero vamos a reordenar la información de las columnas

1. Eliminamos la primera columna que no aporta nada. 

In [35]:
#eliminar columna 'ID Proveedor'
#prov_productos.drop(prov_productos.columns[0], axis=1, inplace=True)

In [36]:
prov_productos.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,1,252,Reckitt Benckiser Argentina S.A.,1,Almacn,4,Limpieza,3,Ba¤o y Hogar,4,Desodorantes de ambientes,1,Desodorantes en aerosol,27003,273920,REFILL CITRICO OIL ESCENCE AIR WICK CITRICOS ...,1,2019-07-12 11:04:07.967
1,1,252,Reckitt Benckiser Argentina S.A.,1,Almacn,4,Limpieza,3,Ba¤o y Hogar,4,Desodorantes de ambientes,1,Desodorantes en aerosol,27004,273921,FRESHMATIC AIRWICK LAVANDA REPUESTO 250.00 MLT,1,2019-07-12 11:04:07.967
2,1,252,Reckitt Benckiser Argentina S.A.,1,Almacn,4,Limpieza,3,Ba¤o y Hogar,4,Desodorantes de ambientes,1,Desodorantes en aerosol,27005,284012,AIR WICK CITRICO FULL REP.GRATIS AIR WICK AIR ...,1,2019-07-12 11:04:07.967
3,1,252,Reckitt Benckiser Argentina S.A.,1,Almacn,4,Limpieza,3,Ba¤o y Hogar,4,Desodorantes de ambientes,1,Desodorantes en aerosol,27012,359572,MAGNOLIA Y CHERRY AIR WICK REPUESTO 175.00 GR,1,2019-07-12 11:04:07.967
4,1,252,Reckitt Benckiser Argentina S.A.,1,Almacn,4,Limpieza,3,Ba¤o y Hogar,4,Desodorantes de ambientes,1,Desodorantes en aerosol,27007,415268,FRESHMATIC BOSQUE MµGICO APARATO AIR WICK LUSH...,1,2019-07-12 11:04:07.967


### Eliminamos la columna 0 y 11                 

In [37]:
prov_productos = prov_productos.drop(prov_productos.columns[[0, 11]], axis=1)


In [38]:
prov_productos.head(5)

,1,2,3,4,5,6,7,8,9,10,12,13,14,15,16,17
0,252,Reckitt Benckiser Argentina S.A.,1,Almacn,4,Limpieza,3,Ba¤o y Hogar,4,Desodorantes de ambientes,Desodorantes en aerosol,27003,273920,REFILL CITRICO OIL ESCENCE AIR WICK CITRICOS ...,1,2019-07-12 11:04:07.967
1,252,Reckitt Benckiser Argentina S.A.,1,Almacn,4,Limpieza,3,Ba¤o y Hogar,4,Desodorantes de ambientes,Desodorantes en aerosol,27004,273921,FRESHMATIC AIRWICK LAVANDA REPUESTO 250.00 MLT,1,2019-07-12 11:04:07.967
2,252,Reckitt Benckiser Argentina S.A.,1,Almacn,4,Limpieza,3,Ba¤o y Hogar,4,Desodorantes de ambientes,Desodorantes en aerosol,27005,284012,AIR WICK CITRICO FULL REP.GRATIS AIR WICK AIR ...,1,2019-07-12 11:04:07.967
3,252,Reckitt Benckiser Argentina S.A.,1,Almacn,4,Limpieza,3,Ba¤o y Hogar,4,Desodorantes de ambientes,Desodorantes en aerosol,27012,359572,MAGNOLIA Y CHERRY AIR WICK REPUESTO 175.00 GR,1,2019-07-12 11:04:07.967
4,252,Reckitt Benckiser Argentina S.A.,1,Almacn,4,Limpieza,3,Ba¤o y Hogar,4,Desodorantes de ambientes,Desodorantes en aerosol,27007,415268,FRESHMATIC BOSQUE MµGICO APARATO AIR WICK LUSH...,1,2019-07-12 11:04:07.967


In [39]:
prov_productos.shape

(82, 16)

2. Renombramos las columnas

In [40]:
prov_productos.columns = [
    'ID Proveedor', 
    'Razon Social Proveedor', 
    'ID Seccion', 
    'Descripcion Seccion', 
    'ID Gran Familia', 
    'Descripcion Gran Familia', 
    'ID Familia', 
    'Descripcion Familia', 
    'ID Subfamilia', 
    'Descripcion Sub Familia',
    'Descripcion Articulo', 
    'ID Articulo',
    'ID Producto', 
    'Descripcion Producto', 
    'ID Región',
    'Ultima Fecha'  # Añadimos el nombre para la columna adicional que faltaba y que no está en los doc.
]

In [41]:
prov_productos.shape

(82, 16)

In [42]:
prov_productos.head(5)

,ID Proveedor,Razon Social Proveedor,ID Seccion,Descripcion Seccion,ID Gran Familia,Descripcion Gran Familia,ID Familia,Descripcion Familia,ID Subfamilia,Descripcion Sub Familia,Descripcion Articulo,ID Articulo,ID Producto,Descripcion Producto,ID Región,Ultima Fecha
0,252,Reckitt Benckiser Argentina S.A.,1,Almacn,4,Limpieza,3,Ba¤o y Hogar,4,Desodorantes de ambientes,Desodorantes en aerosol,27003,273920,REFILL CITRICO OIL ESCENCE AIR WICK CITRICOS ...,1,2019-07-12 11:04:07.967
1,252,Reckitt Benckiser Argentina S.A.,1,Almacn,4,Limpieza,3,Ba¤o y Hogar,4,Desodorantes de ambientes,Desodorantes en aerosol,27004,273921,FRESHMATIC AIRWICK LAVANDA REPUESTO 250.00 MLT,1,2019-07-12 11:04:07.967
2,252,Reckitt Benckiser Argentina S.A.,1,Almacn,4,Limpieza,3,Ba¤o y Hogar,4,Desodorantes de ambientes,Desodorantes en aerosol,27005,284012,AIR WICK CITRICO FULL REP.GRATIS AIR WICK AIR ...,1,2019-07-12 11:04:07.967
3,252,Reckitt Benckiser Argentina S.A.,1,Almacn,4,Limpieza,3,Ba¤o y Hogar,4,Desodorantes de ambientes,Desodorantes en aerosol,27012,359572,MAGNOLIA Y CHERRY AIR WICK REPUESTO 175.00 GR,1,2019-07-12 11:04:07.967
4,252,Reckitt Benckiser Argentina S.A.,1,Almacn,4,Limpieza,3,Ba¤o y Hogar,4,Desodorantes de ambientes,Desodorantes en aerosol,27007,415268,FRESHMATIC BOSQUE MµGICO APARATO AIR WICK LUSH...,1,2019-07-12 11:04:07.967


In [43]:
prov_productos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   ID Proveedor              82 non-null     int64 
 1   Razon Social Proveedor    82 non-null     object
 2   ID Seccion                82 non-null     int64 
 3   Descripcion Seccion       82 non-null     object
 4   ID Gran Familia           82 non-null     int64 
 5   Descripcion Gran Familia  82 non-null     object
 6   ID Familia                82 non-null     int64 
 7   Descripcion Familia       82 non-null     object
 8   ID Subfamilia             82 non-null     int64 
 9   Descripcion Sub Familia   82 non-null     object
 10  Descripcion Articulo      82 non-null     object
 11  ID Articulo               82 non-null     int64 
 12  ID Producto               82 non-null     int64 
 13  Descripcion Producto      82 non-null     object
 14  ID Región                 82

Observamos que la útlima columna 'Ultima Fecha' está en formato object, debería de ser en formato fecha.

In [48]:
prov_productos['Ultima Fecha'] = pd.to_datetime(prov_productos['Ultima Fecha'], errors='coerce')

In [49]:
prov_productos.dtypes

ID Proveedor                         int64
Razon Social Proveedor              object
ID Seccion                           int64
Descripcion Seccion                 object
ID Gran Familia                      int64
Descripcion Gran Familia            object
ID Familia                           int64
Descripcion Familia                 object
ID Subfamilia                        int64
Descripcion Sub Familia             object
Descripcion Articulo                object
ID Articulo                          int64
ID Producto                          int64
Descripcion Producto                object
ID Región                            int64
Ultima Fecha                datetime64[ns]
dtype: object

Comprobamos que se ha convertido correctamente la columna

In [51]:
prov_productos['Ultima Fecha'].isna()

0     False
1     False
2     False
3     False
4     False
      ...  
77    False
78    False
79    False
80    False
81    False
Name: Ultima Fecha, Length: 82, dtype: bool

In [52]:
prov_productos['Ultima Fecha']

0    2019-07-12 11:04:07.967
1    2019-07-12 11:04:07.967
2    2019-07-12 11:04:07.967
3    2019-07-12 11:04:07.967
4    2019-07-12 11:04:07.967
               ...          
77   2019-07-12 11:04:07.967
78   2019-07-12 11:04:07.967
79   2019-07-12 11:04:07.967
80   2019-07-12 11:04:07.967
81   2019-07-12 11:04:07.967
Name: Ultima Fecha, Length: 82, dtype: datetime64[ns]

In [47]:
prov_productos.isnull().sum()

ID Proveedor                0
Razon Social Proveedor      0
ID Seccion                  0
Descripcion Seccion         0
ID Gran Familia             0
Descripcion Gran Familia    0
ID Familia                  0
Descripcion Familia         0
ID Subfamilia               0
Descripcion Sub Familia     0
Descripcion Articulo        0
ID Articulo                 0
ID Producto                 0
Descripcion Producto        0
ID Región                   0
Ultima Fecha                0
dtype: int64

In [53]:
prov_productos

,ID Proveedor,Razon Social Proveedor,ID Seccion,Descripcion Seccion,ID Gran Familia,Descripcion Gran Familia,ID Familia,Descripcion Familia,ID Subfamilia,Descripcion Sub Familia,Descripcion Articulo,ID Articulo,ID Producto,Descripcion Producto,ID Región,Ultima Fecha
0,252,Reckitt Benckiser Argentina S.A.,1,Almacn,4,Limpieza,3,Ba¤o y Hogar,4,Desodorantes de ambientes,Desodorantes en aerosol,27003,273920,REFILL CITRICO OIL ESCENCE AIR WICK CITRICOS ...,1,2019-07-12 11:04:07.967
1,252,Reckitt Benckiser Argentina S.A.,1,Almacn,4,Limpieza,3,Ba¤o y Hogar,4,Desodorantes de ambientes,Desodorantes en aerosol,27004,273921,FRESHMATIC AIRWICK LAVANDA REPUESTO 250.00 MLT,1,2019-07-12 11:04:07.967
2,252,Reckitt Benckiser Argentina S.A.,1,Almacn,4,Limpieza,3,Ba¤o y Hogar,4,Desodorantes de ambientes,Desodorantes en aerosol,27005,284012,AIR WICK CITRICO FULL REP.GRATIS AIR WICK AIR ...,1,2019-07-12 11:04:07.967
3,252,Reckitt Benckiser Argentina S.A.,1,Almacn,4,Limpieza,3,Ba¤o y Hogar,4,Desodorantes de ambientes,Desodorantes en aerosol,27012,359572,MAGNOLIA Y CHERRY AIR WICK REPUESTO 175.00 GR,1,2019-07-12 11:04:07.967
4,252,Reckitt Benckiser Argentina S.A.,1,Almacn,4,Limpieza,3,Ba¤o y Hogar,4,Desodorantes de ambientes,Desodorantes en aerosol,27007,415268,FRESHMATIC BOSQUE MµGICO APARATO AIR WICK LUSH...,1,2019-07-12 11:04:07.967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,1550,Clorox argentina s.a.,1,Almacn,4,Limpieza,3,Ba¤o y Hogar,4,Desodorantes de ambientes,Desodorantes en aerosol,4093,454775,POETT AMBIENTAL ESPIRITU JOVEN NUEVO D 0.377 KG,1,2019-07-12 11:04:07.967
78,1550,Clorox argentina s.a.,1,Almacn,4,Limpieza,3,Ba¤o y Hogar,4,Desodorantes de ambientes,Desodorantes en aerosol,4094,455722,POETT AMBIENTAL DEJA VU NUEVO D 0.377 KG,1,2019-07-12 11:04:07.967
79,1550,Clorox argentina s.a.,1,Almacn,4,Limpieza,3,Ba¤o y Hogar,4,Desodorantes de ambientes,Desodorantes en aerosol,4096,457746,POETT AMBIENTAL BOSQUE DE BAMBU NUEVO D 0.377 KG,1,2019-07-12 11:04:07.967
80,1550,Clorox argentina s.a.,1,Almacn,4,Limpieza,3,Ba¤o y Hogar,4,Desodorantes de ambientes,Desodorantes en aerosol,4099,489517,POETT POETT AF360 LOVE EVOLUTION POETT AF360 L...,1,2019-07-12 11:04:07.967


In [55]:
prov_productos.duplicated().sum()

0

Guardamos en formato csv

In [56]:
# Guardar el DataFrame en un archivo CSV
prov_productos.to_csv('prov_productos_clean.csv', index=False)
